In [1]:
import requests
import re
from bs4 import BeautifulSoup
import csv

In [8]:
#configurations
base_url_prefix = "https://www.linkedin.com/jobs/search?keywords=software%20engineer%20OR%20engineering%20manager&location=United%20States&pageNum=0&start="
row_increment_default = 25
max_row_default = 51
debug_mode = False
debug_company=""

In [9]:
#initializations
jobs={}
seen_jobs={}

In [10]:
for i in range(0,max_row_default,row_increment_default):
    base_url = f"{base_url_prefix}{i}"
    response = requests.get(base_url)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        job_listings = soup.find_all('div', {'class':'job-search-card'})
        for job in job_listings:
            try:
                date = job.find('time', {'class':'job-search-card__listdate'}).attrs['datetime']
            except AttributeError:
                date = job.find('time', {'class':'job-search-card__listdate--new'}).attrs['datetime']
            title = job.find('h3', {'class': 'base-search-card__title'}).text.strip()
            company = job.find('a', {'class': 'hidden-nested-link'}).text.strip()
            location = job.find('span', {'class': 'job-search-card__location'}).text.strip()
            link = job.find('a', {'class': 'base-card__full-link'}).attrs['href']
            pattern = r"(.*-)(\d+)"
            job_id = re.search(pattern,link).group(2)

            #check cache; skip if already seen
            if seen_jobs.get(job_id,'') != '':
                if debug_mode:
                    print(f"we've seen {job_id}")
                continue
            else:
                seen_jobs[job_id]=True
                
            #test
            if debug_mode and company != debug_company:
                continue
            elif debug_mode:
                print(f"title:{title}, id:{sanitized_link}, date:{date}")
    
            #initialize kv entry
            if company not in jobs:
                jobs.setdefault(company,{})
                jobs[company]={'eng':0,'em':0,'maxDate':''}
                
            #increment count
            if 'Manager' not in title:
                jobs[company]['eng']=jobs.get(company,{}).get('eng',0)+1
            else:
                jobs[company]['em']=jobs.get(company,{}).get('em',0)+1
                
            #set max date
            jobs[company]['maxDate'] = max(jobs[company]['maxDate'], date)

    else:
        print("Failed to fetch job listings.")
        break
print(f"jobs found:{len(jobs)}")

jobs=dict(sorted(jobs.items(),key=lambda x:x[1]['eng']+x[1]['em'], reverse=True))
filtered_jobs={k:v for k,v in jobs.items() if v['eng']+v['em']>1}
print('\n'.join([f"{key}: {value}" for key, value in filtered_jobs.items()]))
#print('\n'.join([f"{key}: {value}" for key, value in seen_jobs.items()]))

jobs found:47
Google: {'eng': 13, 'em': 0, 'maxDate': '2024-05-17'}
The Walt Disney Company: {'eng': 2, 'em': 0, 'maxDate': '2024-05-18'}
Nike: {'eng': 2, 'em': 0, 'maxDate': '2024-05-17'}
LinkedIn: {'eng': 2, 'em': 0, 'maxDate': '2024-05-14'}
Ever: {'eng': 2, 'em': 0, 'maxDate': '2024-05-16'}
Midpoint Markets: {'eng': 2, 'em': 0, 'maxDate': '2024-05-11'}
Genesis Therapeutics: {'eng': 2, 'em': 0, 'maxDate': '2024-04-22'}


In [ ]:
#todo connect to google drive via api